In [ ]:
from prefect.engine.executors import LocalExecutor

# ... your flow construction

state = flow.run(executor=LocalExecutor()) # <-- the executor needs to be initialized


In [ ]:
from prefect import Flow, task
from prefect.utilities.debug import raise_on_exception


@task
def div(x):
    return 1 / x

with Flow() as f:
    val = div(0)
    
with raise_on_exception():
    f.run()

---------------------------------------------------------------------------

ZeroDivisionError                         Traceback (most recent call last)

<ipython-input-1-82c40dd24406> in <module>()
     11 
     12 with raise_on_exception():
---> 13     f.run()


... # the full traceback is long

<ipython-input-1-82c40dd24406> in div(x)
      5 @task
      6 def div(x):
----> 7     return 1 / x
      8 
      9 with Flow() as f:


ZeroDivisionError: division by zero


In [ ]:
from prefect import Flow, task


@task
def gotcha():
    tup = ('a', ['b'])
    try:
        tup[1] += ['c']
    except TypeError:
        assert len(tup[1]) == 1
        

flow = Flow(tasks=[gotcha])
state = flow.run(return_failed=True)
state.result # {<Task: gotcha>: Failed("")}

failed_state = state.result[gotcha]
raise failed_state.result

---------------------------------------------------------------------------

TypeError                                 Traceback (most recent call last)

<ipython-input-50-8efcdf8dacda> in gotcha()
      7     try:
----> 8         tup[1] += ['c']
      9     except TypeError:


TypeError: 'tuple' object does not support item assignment


During handling of the above exception, another exception occurred:


AssertionError                            Traceback (most recent call last)

<ipython-input-50-8efcdf8dacda> in <module>()
     16 
     17 failed_state = state.result[gotcha]
---> 18 raise failed_state.result

...

<ipython-input-50-8efcdf8dacda> in gotcha()
      8         tup[1] += ['c']
      9     except TypeError:
---> 10         assert len(tup[1]) == 1
     11 
     12 


AssertionError: 

%debug # using the IPython magic method to start a pdb session


In [ ]:
@task
def fixed():
    tup = ('a', ('b'))
    try:
        tup[1] += ('c')
    except TypeError:
        assert len(tup[1]) == 1
        
        
broken = flow.get_tasks(name="gotcha")[0]
flow.replace(broken, fixed)

flow.run() # Success("All reference tasks succeeded.")
